In [1]:
#import of packages

import numpy as np
import math
import matplotlib.pyplot as plt

from scipy.linalg import sqrtm #matrix square root
from scipy.stats import gaussian_kde

print("ok")

ok


In [6]:
def plots_work():
  return "Plotting functions integrated."

In [15]:
#plotting functions

#time series of the eigenvalues of the rv

def plot_eigenvalues_rv(eigenvalues_rv, true_eigenvalues_Sigma, title="time series of the eigenvalues of the rv", save_image=0,
                        name_of_copy="plot_eigenvalues_rv.png"):

    """
    Plots the largest four eigenvalues of the realized variance and
    the mean of the eigenvalues without the three largest of them.
    In addition, the true eigenvalues of the QV are marked.

    Parameters
    ----------
    eigenvalues_rv : numpy.ndarray
        Matrix of dimension paths times d. It contains in the j-th column the j-th largest eigenvalues of the rv.
    true_eigenvalues_Sigma : numpy.ndarray
        Vector containing the true eigenvalues of Sigma.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    eigenvalue_mean_rv_without_three_largest=np.mean(eigenvalues_rv[:, 3:], axis=1)

    plt.figure(figsize=(8, 4))
    plt.plot(eigenvalues_rv[:, 0], label='ev_1')
    plt.plot(eigenvalues_rv[:, 1], label='ev_2')
    plt.plot(eigenvalues_rv[:, 2], label='ev_3')
    plt.plot(eigenvalues_rv[:, 3], label='ev_4')
    plt.plot(eigenvalue_mean_rv_without_three_largest, label='mean_without_three_largest_ev')

    plt.axhline(y=true_eigenvalues_Sigma[0], linestyle="-", label="true_ev_1", color="black")
    plt.axhline(y=true_eigenvalues_Sigma[1], linestyle="--", label="true_ev_2", color="black")
    plt.axhline(y=true_eigenvalues_Sigma[2], linestyle="-.", label="true_ev_3", color="black")
    plt.axhline(y=true_eigenvalues_Sigma[3], linestyle=":", label="true_ev_4", color="black")
    plt.axhline(y=np.mean(true_eigenvalues_Sigma[3:]), linestyle=(0, (1, 8)), label="true_mean_without_three_largest", color="black")

    y_lim=np.max(eigenvalues_rv[:, 0])+1
    plt.yticks(np.arange(0,y_lim))

    plt.title(title)
    plt.xlabel('simulations')
    plt.ylabel('eigenvalue')
    plt.legend(loc='upper right', title='legend')
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()


#plot of the means of the eigenvalues of the rv

def plot_means_of_eigenvalues(eigenvalues, true_eigenvalues_Sigma, title="means of the eigenvalues of the rv", save_image=0,
                        name_of_copy="plot_mean_eigenvalues_rv.png"):

    """
    Plots the means of the eigenvalues of an estimator and the true_eigenvalues of the QV.

    Parameters
    ----------
    eigenvalues : numpy.ndarray
        Matrix of dimension paths times d. It contains in the j-th column the j-th largest eigenvalues.
    true_eigenvalues_Sigma : numpy.ndarray
        Vector containing the true eigenvalues of Sigma.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    # Compute mean of each column
    eigenvalue_means = np.mean(eigenvalues, axis=0)

    cols= np.arange(1, d+1)
    #plot
    plt.figure(figsize=(8, 4))
    plt.bar(cols, eigenvalue_means, color='skyblue', edgecolor='black')

    width=0.8
    for k in range(len(true_eigenvalues_Sigma)):
        plt.hlines(y=true_eigenvalues_Sigma[k], xmin=cols[k]-width/2, xmax=cols[k]+width/2, colors="black", linestyle="--" )


    plt.xlabel('eigenvalue')
    plt.ylabel('mean')
    plt.title(title)
    plt.xticks(cols) #ticks at the columns
    plt.yticks(np.arange(0,np.max(eigenvalue_means)+1))

    plt.grid(axis='y', linestyle='--', alpha=0.7) #only horizontal lines
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()


#kernel density estimation of lambda_subsam

def kde_of_lambdas_half(lambdas_used, smoothing=0.1, title="kernel density estimation of the lambda_subsam/2", save_image=0,
                        name_of_copy="kde_of_half_of_lambda_subsam.png"):

    """
    Plots the kernel desnity estimation of the half of the used tuning parameters,
    which is the shrinking amount of the eigenvalues.

    Parameters
    ----------
    lambdas_used : numpy.ndarray
        Vector containing the used lambdas.
    smoothing : float
        How smooth is the kde.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    kde = gaussian_kde(lambdas_used/2, bw_method=smoothing) #bw_method controls smoothing
    x = np.linspace(np.min(lambdas_used/2)*0.95, np.max(lambdas_used/2)*1.05, 1000)
    plt.plot(x, kde(x))
    plt.title(title)
    plt.xlabel('lambda/2')
    plt.ylabel('density estimate')

    plt.xticks()

    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()

#histogram of the rank of teh PRV

def histogram_of_the_rank(ranks, true_rank_of_denoised_Sigma,
                          title="rank of the PRV estimator",
                          save_image=0, name_of_copy="histogram_rank_prv.png"):

    """
    Creates a histogram that shows the realtive frequency of a rank of an estimator.

    Parameters
    ----------
    ranks : numpy.ndarray
        Vector containing the ranks of an estimator in all simulations.
    true_rank_of_denoised_Sigma : int
        Rank of Sigma_denoised, which is the number of dominant eigenvalues + the number of normal eigenvalues.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """
    #rank of prv

    bins_def = np.arange(np.min(ranks) -2, np.max(ranks) + 3) - 0.5      #bin edges:-0.5,0.5,...,max_rank+2-0.5
    plt.hist(ranks, bins=bins_def, density=True, edgecolor='black')     #density for relative

    plt.axvline(x=true_rank_of_denoised_Sigma,color="black",linestyle="--",linewidth=1.5)
    if max(np.max(ranks), true_rank_of_denoised_Sigma)-min(np.min(ranks),true_rank_of_denoised_Sigma) <10:
        plt.xticks(np.arange(min(np.min(ranks) -2, true_rank_of_denoised_Sigma) ,
                             max(np.max(ranks) + 3, true_rank_of_denoised_Sigma)))
    else:
        ax = plt.gca() #actual axes
        auto_ticks = ax.get_xticks() #the automatically generated ticks
        ticks = np.unique(np.append(auto_ticks, true_rank_of_denoised_Sigma)) #add true_rank_of_denoised_Sigma
        labels_ticks = [f"{true_rank_of_denoised_Sigma}" if t == true_rank_of_denoised_Sigma else f"{t:g}" for t in ticks]
        #labels as t without zeros ({t:g})
        plt.xticks(ticks, labels_ticks)

    improved_title=title + f' in case of {true_rank_of_denoised_Sigma} driving factors'
    plt.title(improved_title)
    plt.xlabel("rank")
    plt.ylabel("relative frequency")
    plt.grid(True)

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()



#boxplots of the estimation errors

def boxplot_estimation_errors(all_errors, title="boxplot of estimation errors of the QV (means, no medians)", save_image=0,
                              name_of_copy="boxplot_est_errors.png"):

    """
    Creates a boxplot of the estimation errors.

    Parameters
    ----------
    all_errors : numpy.ndarray
        Matrix of dimension paths times error_types. It contains in each column the values of a specific error type.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    #define maximal y value on the y-axis:
    y_max=np.max(np.mean(all_errors, axis=0))*1.25

    #boxplots of the errors

    data=[all_errors[:,0], all_errors[:,1], all_errors[:,2], all_errors[:,3], all_errors[:,4], all_errors[:,5],
          all_errors[:,6], all_errors[:,7], all_errors[:,8], all_errors[:,9], all_errors[:,10], all_errors[:,11] ]

    labels=["2-norm^2 rv error", "2-norm^2 prv error",
            "1-norm rv error", "1-norm prv error",
            "spectral-norm rv error", "spectral-norm prv error",
            "denoised-2-norm^2 rv error", "denoised-2-norm^2 prv error",
            "denoised-1-norm rv error", "denoised-1-norm prv error",
            "denoised-spectral-norm rv error", "denoised-spectral-norm prv error"]

    colors = plt.cm.viridis(np.linspace(0.2, 0.95, len(data))) # len(data) evenly spaced numbers

    plt.figure(figsize=(8, 8))

    # create boxplot
    box = plt.boxplot(
        data,
        patch_artist=True, #colors for boxes
        notch=False, #no notches
        showmeans=True, #means
        meanline=True, #as lines
        showfliers=False #no outliers: then you need False
    )

    #apply colors
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_edgecolor('black')
        patch.set_linewidth(1.2)

    #means
    for mean in box['means']:
        mean.set(color='black', linewidth=1.5)

    #no medians
    for median in box['medians']:
        median.set_visible(False)
        #median.set(color='blue', linewidth=1.5)

    # x-axis
    plt.xticks(
        ticks=range(1, len(data)+1),
        labels=labels,
        rotation=60,
        ha='right' #end of labels at the tick
    )

    #y-axis
    plt.yticks(np.arange(0,y_max,1))

    plt.grid(axis='y', linestyle='--', alpha=0.5) #alpha for transparency
    plt.ylabel("errors", fontsize=12)
    plt.ylim(0,y_max)

    #title
    plt.title(title, fontsize=16)

    plt.tight_layout()
    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()


#plot time series for sensitivity analysis of L_lambda

def plot_time_series_sensitivity_analysis_L_lambda(time_series_matrix, L_lambdas, number_of_observations, name_y_axis="lambda_subsam/2",
                                                   title="influence of L_Lambda on the tuning parameter lambda_subsam/2",
                                                   y_ticks_automatic=0, save_image=0, name_of_copy="plot_lambda_subsam_sensitive_L_lambda.png"):

    """
    Plots the different values that depend on L_lambda in several time series.

    Parameters
    ----------
    time_series_matrix : numpy.ndarray
        Matrix of dimension paths times number of different L_lambdas.
    L_lambdas : numpy.ndarray
        Vector containing the different L_lambdas.
    number_of_observations : int
        How many observations are made on one path.
    name_y_axis : str
        Label of y axis.
    title : str
        Title of the plot.
    y_ticks_automatic : int
        The value 0 means: not automatic. The value 1 means: automatic.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    plt.figure(figsize=(8, 4))
    for z in range(len(L_lambdas)):
        plt.plot(time_series_matrix[:, z], label=f'L_lambda={L_lambdas[z]}')

    if y_ticks_automatic==0:
        y_lim=np.max(time_series_matrix)+1
        plt.yticks(np.arange(0,y_lim))
    else:
        plt.yticks()

    title_new=title + f" for n={number_of_observations}"
    plt.title(title_new)
    plt.xlabel('simulations')
    plt.ylabel(name_y_axis)
    plt.legend(loc='upper right', title='legend')
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()


#plot time series for sensitivity analysis of lambda

def plot_time_series_sensitivity_analysis_lambda_prefactors(time_series_matrix, lambda_prefactors, number_of_observations,
                                                            name_y_axis="rank" ,
                                                            title="influence of the tuning parameter lambda on the rank of the PRV",
                                                            y_ticks_automatic=0, save_image=0,
                                                            name_of_copy="plot_rank_PRV_sensitive_lambda.png"):

    """
    Plots the different values that depend on prefactor of lambda_subsam in several time series.

    Parameters
    ----------
    time_series_matrix : numpy.ndarray
        Matrix of dimension paths times number of different lambdas.
    lambda_prefactors : numpy.ndarray
        Vector containing the different prefactors of lambda_subsam.
    number_of_observations : int
        How many observations are made on one path.
    name_y_axis : str
        Label of y axis.
    title : str
        Title of the plot.
    y_ticks_automatic : int
        The value 0 means: not automatic. The value 1 means: automatic.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    plt.figure(figsize=(8, 4))
    for z in range(len(lambda_prefactors)):
        plt.plot(time_series_matrix[:, z], label=f'lambda prefactor={lambda_prefactors[z]}')

    if y_ticks_automatic==0:
        y_lim=np.max(time_series_matrix)+1
        plt.yticks(np.arange(0,y_lim))
    else:
        plt.yticks()

    title_new=title + f" for n={number_of_observations}"
    plt.title(title_new)
    plt.xlabel('simulations')
    plt.ylabel(name_y_axis)
    plt.legend(loc='upper right', title='legend')
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()

#plot mean errors for sensitivity analysis of lambda

def plot_mean_of_errors_sensitivity_analysis_lambda_prefactors(time_series_matrix, lambda_prefactors, dimension=30, number_of_observations=78,
                                                            name_y_axis="error" ,
                                                            title="influence of the tuning parameter lambda on the error means of the PRV",
                                                            y_ticks_automatic=0, save_image=0,
                                                            name_of_copy="scatter_plot_error_means_PRV_sensitive_lambda.png"):

    """
    Plots the error means for specific prefactors of lambda_subsam.

    Parameters
    ----------
    time_series_matrix : numpy.ndarray
        Matrix of dimension paths times number of different lambdas.
    lambda_prefactors : numpy.ndarray
        Vector containing the different prefactors of lambda_subsam.
    dimesnion : int
      Dimension d of the log-prices.
    number_of_observations : int
        How many observations are made on one path.
    name_y_axis : str
        Label of y axis.
    title : str
        Title of the plot.
    y_ticks_automatic : int
        The value 0 means: not automatic. The value 1 means: automatic.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    plt.figure(figsize=(8, 4))

    means=np.mean(time_series_matrix, axis=0)  #means over the columns

    plt.scatter(lambda_prefactors, means )

    if y_ticks_automatic==0:
        y_lim=np.max(time_series_matrix)+1
        plt.yticks(np.arange(0,y_lim))
    else:
        plt.yticks()

    title_new=title + f" for n={number_of_observations} and d={dimension}"
    plt.title(title_new)
    plt.xlabel('prefactors of lambda_subsam')
    plt.ylabel(name_y_axis)
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()


#time series of different tuning parameters

def plot_lambda_choices(lambda_subsam, lambda_esterr, lambda_conint,
                        title="time series of different choices for the tuning parameter",
                                                      save_image=0,
                                                      name_of_copy="plot_different_lambda_choices.png"):

    """
    Plots the time series of different choices of lambdas.

    Parameters
    ----------
    lambda_subsam : numpy.ndarray
        Vector containing the tuning parameters lambda_subsam for all paths.
    lambda_esterr : float
        The tuning poarameter lambda_esterr.
    lambda_conint : float
        The tuning poarameter lambda_conint.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    plt.figure(figsize=(8, 4))
    mean_subsam=np.mean(lambda_subsam)
    plt.plot(lambda_subsam, label=f'lambda_subsam with mean ≈ {round(mean_subsam,3)}')
    plt.axhline(y=lambda_esterr, linestyle="-", label=f'lambda_esterr ≈ {round(lambda_esterr,3)}', color="black")
    plt.axhline(y=lambda_conint, linestyle="-", label=f'lambda_conint ≈ {round(lambda_conint,3)}', color="red")

    plt.yticks()
    #y - axis logarithmic
    #plt.yscale("log")

    plt.title(title)
    plt.xlabel('simulations')
    plt.ylabel('tuning parameter')
    plt.legend(loc='best', title='legend')
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()
    '''
    fig, ax1=plt.subplots()
    ax1.plot(squared_frobenius_errors, label='squared Forbenius norm error')

    ax2=ax1.twinx()
    ax2.axhline(y=bound, linestyle="-", label=f'bound with lambda={lambda_esterr}', color="black")

    plt.show()
    plt.close()
    '''

#time series of the squared frobenius error and its bound from Theorem 4.8

def plot_bound_of_squared_frobenius_error_theorem_4_8(squared_frobenius_errors, bound, lambda_esterr,
                                                      title="time series of squared frobenius norm error and its bound",
                                                      save_image=0,
                                                      name_of_copy="plot_error_bound_theorem_4_8.png"):

    """
    Plots the squared Frobenius error of the PRV with its bound from Theorem 4.8.

    Parameters
    ----------
    squared_frobenius_errors : numpy.ndarray
        Vector containing the squared Frobenius errors of the PRV for all paths.
    bound : float
        Error bound from Theorem 4.8.
    lambda_esterr : float
        The tuning poarameter lambda_esterr used in the PRV and the bound.
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    plt.figure(figsize=(8, 4))
    mean_error=np.mean(squared_frobenius_errors)
    plt.plot(squared_frobenius_errors, label=f'error with mean ≈ {round(mean_error,3)}')
    plt.axhline(y=bound, linestyle="-", label=f'bound at ≈ {round(bound,3)}', color="black")

    plt.yticks()
    #y - axis logarithmic
    #plt.yscale("log")

    plt.title(title+f' with lambda≈ {round(lambda_esterr,3)}')
    plt.xlabel('simulations')
    plt.ylabel('squared error')
    plt.legend(loc='best', title='legend')
    plt.grid(True)
    plt.tight_layout()

    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.show()
    plt.close()
    '''
    fig, ax1=plt.subplots()
    ax1.plot(squared_frobenius_errors, label='squared Forbenius norm error')

    ax2=ax1.twinx()
    ax2.axhline(y=bound, linestyle="-", label=f'bound with lambda={lambda_esterr}', color="black")

    plt.show()
    plt.close()
    '''


#boxplots of the eigenvalues and confidence intervals
def boxplot_eigenvalues_confidence(eigenvalues_prv, indices_of_plotted_eigenvalues, Sigma, lambda_used,
                               title=f'eigenvalues of the PRV and corresponding confidence intervals',
                               save_image=0,
                               name_of_copy="plot_eigenvalues_confidence_theorem_5_3.png"):

    """
    Creates a boxplot of some eigenvalues of the PRV and marks the confidence interval boundaries from Theorem 5.3.

    Parameters
    ----------
    eigenvalues_prv : numpy.ndarray
        Matrix of dimension paths times d. It contains in each row the eigenvalues of a PRV estimator for a specific path.
    indices_of_plotted_eigenvalues : numpy.ndarray
        Vector containing the positions of the eigenvalues plotted in the boxplot.
    Sigma : numpy.ndarray
        True quadratic variation.
    lambda_used : float
        Tuning parameter used for the calculation of the PRV (typically lambda_conint).
    title : str
        Title of the plot.
    save_image : int
        The value 0 means: do not save. The value 1 means: save the plot.
    name_of_copy : str
        File name of the saved image.

    Returns
    -------
    None

    """

    data=[]
    labels=[]
    for z in range(np.shape(indices_of_plotted_eigenvalues)[0]):
        data.append(eigenvalues_prv[:,indices_of_plotted_eigenvalues[z]-1])
        labels.append(f'eigenvalue {indices_of_plotted_eigenvalues[z]}')
        
    colors = plt.cm.viridis(np.linspace(0.2, 0.95, len(data))) # len(data) evenly spaced numbers
    


    plt.figure(figsize=(8, 8))

    # create boxplot
    box = plt.boxplot(
        data,
        patch_artist=True, #colors for boxes
        notch=False, #no notches
        showmeans=True, #means
        meanline=True, #as lines
        showfliers=False #no outliers: then you need False
        
    )

    #apply colors
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_edgecolor('black')
        patch.set_linewidth(1.2)

    #means
    for mean in box['means']:
        mean.set(color='black', linewidth=1.5)

    #no medians
    for median in box['medians']:
        median.set_visible(False)
        #median.set(color='blue', linewidth=1.5)

    # x-axis
    plt.xticks(
        ticks=range(1, len(data)+1),
        labels=labels,
        rotation=60,
        ha='right' #end of labels at the tick
    )

    #y-axis
    plt.yticks()

    plt.grid(axis='y', linestyle='--', alpha=0.5) #alpha for transparency
    plt.ylabel("eigenvalues", fontsize=12)
    #plt.ylim()

    true_ev_Sigma=np.sort(np.linalg.eigh(Sigma)[0])[::-1] #sort creates increasing, now decreasing

    for z in range(np.shape(indices_of_plotted_eigenvalues)[0]):
        plt.hlines(true_ev_Sigma[indices_of_plotted_eigenvalues[z]-1]+lambda_used, z+1-0.25, z+1+0.25,  linestyle=":", color="red" )
        plt.hlines(max(true_ev_Sigma[indices_of_plotted_eigenvalues[z]-1]-lambda_used,0), z+1-0.25, z+1+0.25,  linestyle=":", color="purple" )
    
    # artifical lines for labels
    plt.plot([], [], linestyle=":", label="upper bounds",  color="red")
    plt.plot([], [], linestyle=":", label="lower bounds",  color="purple")
    
    #title
    plt.title(title, fontsize=16)

    plt.tight_layout()
    if save_image==1:
        plt.savefig(name_of_copy, dpi=300, bbox_inches="tight")
        #high resolution dpi, cut of edge bbox

    plt.legend(loc='center right', title='legend')
    
    plt.show()
    plt.close()

print("ok")



ok
